In [8]:
from bs4 import BeautifulSoup
import requests
import time
from datetime import datetime
import pandas as pd

In [9]:
def get_search_result(nIdx, sPID, sAddr):
    now = datetime.now()  # current date and time
    date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

    url = 'https://www.google.com/search'
    headers = {
        'Accept': '*/*',
        'Accept-Language': 'en-US,en;q=0.5',
        'accept-encoding': 'gzip,deflate,br',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36',
    }
    headers['Referer'] = sAddr

    try:
        parameters = {'q': sAddr}
        response = requests.get(url, headers=headers, params = parameters)

        resp_code = response.status_code
        resp_text = response.text
        #print(resp_code, resp_text)

        if 'name="robots"' in resp_text:
            print(f"[DEBUG] CAPTCHA")
            sys.exit()

        #print(f"[DEBUG] {resp_code} {resp_text}")

        soup = BeautifulSoup(resp_text, 'html.parser')
        search = soup.find(id = 'search')
        if search:
            first_link = search.find('a')
            sFirstLink = first_link['href']
        else:
            sFirstLink = 'err'

        sFileOut = sPID + "," + sFirstLink + "\n"
        f = open(out_file, "a")
        f.write(sFileOut)
        f.close()

        print(f"[SUCCESS] [{date_time}] PID# {nIdx}: {sAddr}")

    except Exception as e:
        print(f"[FAILURE] [{date_time}] PID# {nIdx}: {sAddr} [ERROR] {e}")

In [10]:
if __name__ == "__main__":

    inp_dir, out_dir = '../data_inp/', '../data_out/'
    inp_file = inp_dir + 'google_zillow_addrs_search_inp_2.csv'
    out_file = out_dir + 'google_zillow_addrs_search_out_2.csv'
    nSleep=5

    df = pd.read_csv(inp_file)
    dct = df.to_dict('index')

    for nIdx in dct.keys():
        #nURLIdx += 1
        sPID  = dct[nIdx]['property_id']
        sAddr = dct[nIdx]['address_str']

        get_search_result(nIdx, sPID, sAddr)

        # nSleep += 2
        # if nSleep>=60: nSleep=10 #Reset sleep time back to 10seconds
        time.sleep(nSleep)

[SUCCESS] [05/01/2023, 10:03:34] PID# 0: zillow-5950-Midnight-Pass-Rd-#309-Siesta-Beach-House-#309-Siesta-Key-FL-34242-United-States
[SUCCESS] [05/01/2023, 10:03:40] PID# 1: zillow-5740-Midnight-Pass-Rd-Unit-F208-Gulf-and-Bay-Club--F208-Siesta-Key-FL-34242-United-States
[SUCCESS] [05/01/2023, 10:03:46] PID# 2: zillow-1010-Point-of-Rocks-Rd-1010-Point-of-Rocks-Rd-(1BR)-Siesta-Key-FL-34242-United-States
[SUCCESS] [05/01/2023, 10:03:52] PID# 3: zillow-103-Mark-St-Serene-Oasis-Destin-FL-32541-United-States
[SUCCESS] [05/01/2023, 10:03:58] PID# 4: zillow-1148-Mulberry-Ct-Mulberry-Ct-1148-Marco-Island-Vacation-Rental-Marco-Island-FL-34145-United-States
[SUCCESS] [05/01/2023, 10:04:03] PID# 5: zillow-222-Spartina-Circle-WC222SP-Santa-Rosa-Beach-FL-32459-United-States
[SUCCESS] [05/01/2023, 10:04:09] PID# 6: zillow-200-Henderson-Resort-Way-Gulf-View-King-318-Destin-FL-32541-United-States
[SUCCESS] [05/01/2023, 10:04:15] PID# 7: zillow-200-Henderson-Resort-Way-Gulf-View-Queen-212-Destin-FL-3254

[SUCCESS] [05/01/2023, 10:10:04] PID# 67: zillow-381-Santa-Rosa-Boulevard-C407-Fort-Walton-Beach-FL-32548-United-States
[SUCCESS] [05/01/2023, 10:10:11] PID# 68: zillow-381-Santa-Rosa-Boulevard-C308-Fort-Walton-Beach-FL-32548-United-States
[SUCCESS] [05/01/2023, 10:10:16] PID# 69: zillow-15100-Emerald-Coast-Parkway-Silver-Shells-St-Lucia-1202-SSLUCIA1202-Destin-Florida-32541-United-States
[SUCCESS] [05/01/2023, 10:10:22] PID# 70: zillow-15300-Emerald-Coast-Parkway-Silver-Shells-St-Croix-1403-SSCROIX1403-Destin-Florida-32541-United-States
[SUCCESS] [05/01/2023, 10:10:28] PID# 71: zillow-4862-Cayview-Avenue-#403-4862-Cayview-Avenue-#403-Orlando-FL-32819-United-States
[SUCCESS] [05/01/2023, 10:10:34] PID# 72: zillow-2002-Majorca-Drive-2002-Majorca-Drive-Kissimmee-FL-34747-United-States
[SUCCESS] [05/01/2023, 10:10:40] PID# 73: zillow-2911-Fable-Street-2911-Fable-Street-Kissimmee-FL-34746-United-States
[SUCCESS] [05/01/2023, 10:10:46] PID# 74: zillow-2622-Dinville-St-2622DV-WH_05-Kissimmee

[SUCCESS] [05/01/2023, 10:16:30] PID# 133: zillow-2914-Gulf-Dr-Dolphin-Cottage-Holmes-Beach-FL-34217-United-States
[SUCCESS] [05/01/2023, 10:16:36] PID# 134: zillow-214-Periwinkle-Plz-Beach-House-Anna-Maria-FL-34216-United-States
[SUCCESS] [05/01/2023, 10:16:41] PID# 135: zillow-1600-Via-De-Luna-Drive-RTW704-Pensacola-Beach-FL-32561-United-States
[SUCCESS] [05/01/2023, 10:16:47] PID# 136: zillow-925-Whelk-Court-VDM8-Fort-Walton-Beach-FL-32548-United-States
[SUCCESS] [05/01/2023, 10:16:53] PID# 137: zillow-2900-Scenic-Highway-98-CR302-Destin-FL-32541-United-States
[SUCCESS] [05/01/2023, 10:16:59] PID# 138: zillow-14511-Perdido-Key-Drive-WM902-Perdido-Key-FL-32507-United-States
[SUCCESS] [05/01/2023, 10:17:04] PID# 139: zillow-520-Richard-Jackson-Blvd-EWV1913-Panama-City-Beach-FL-32407-United-States
[SUCCESS] [05/01/2023, 10:17:10] PID# 140: zillow-4045-W-Co-Highway-30A-#203-SR203-Santa-Rosa-Beach-FL-32459-United-States
[SUCCESS] [05/01/2023, 10:17:16] PID# 141: zillow-416-Morgans-Trail-

[SUCCESS] [05/01/2023, 10:22:57] PID# 200: zillow-272-Blue-Gulf-Drive-RELAXATION-Blue-Mountain-Beach-FL-32459-United-States
[SUCCESS] [05/01/2023, 10:23:03] PID# 201: zillow-112-Los-Angeles-Street-A-GARDEN-COTTAGE-Miramar-Beach-FL-32550-United-States
[SUCCESS] [05/01/2023, 10:23:09] PID# 202: zillow-9900-South-Thomas-Drive-SOP1406-Panama-City-Beach-FL-32408-United-States
[SUCCESS] [05/01/2023, 10:23:14] PID# 203: zillow-1680-E-County-Hwy-30a-Unit-104-VRBO-#578654-WC1680-Santa-Rosa-Beach-FL-32459-United-States
[SUCCESS] [05/01/2023, 10:23:20] PID# 204: zillow-7300-Estero-Blvd-Unit-#1007A-E1007A-Fort-Myers-Beach-FL-33931-United-States
[SUCCESS] [05/01/2023, 10:23:26] PID# 205: zillow-1653-W-County-Hwy-30A-#3120-RED3120-Santa-Rosa-Beach-FL-32459-United-States
[SUCCESS] [05/01/2023, 10:23:32] PID# 206: zillow-1018-Hwy-98-E-#630-J0630-Destin-FL-32541-United-States
[SUCCESS] [05/01/2023, 10:23:38] PID# 207: zillow-2936-Scenic-Gulf-Drive-401-DE-W-MB--Leeward-Key-401-Miramar-Beach-FL-32550-Uni

[SUCCESS] [05/01/2023, 10:29:26] PID# 267: zillow-3501-S-Atlantic-Ave-Unit-803-PDL803-New-Smyrna-Beach-FL-32169-United-States
[SUCCESS] [05/01/2023, 10:29:32] PID# 268: zillow-610-S-Atlantic-Ave-OE-New-Smyrna-Beach-FL-32169-United-States
[SUCCESS] [05/01/2023, 10:29:38] PID# 269: zillow-4501-ESTERO-BLVD-#3-OCEANIC-ESTATE-#3-Fort-Myers-Beach-FL-33931-United-States
[SUCCESS] [05/01/2023, 10:29:44] PID# 270: zillow-6020-Midnight-Pass-Rd-Unit-52N-Aloha-Kai---Unit-52N-Siesta-Key-FL-34242-United-States
[SUCCESS] [05/01/2023, 10:29:50] PID# 271: zillow-814-Forest-Street-TIMELESS-Santa-Rosa-Beach-FL-32459-United-States
[SUCCESS] [05/01/2023, 10:29:56] PID# 272: zillow-510-Gulf-Shore-Drive-R618-Destin-FL-32541-United-States
[SUCCESS] [05/01/2023, 10:30:01] PID# 273: zillow-318-Sand-Hill-St-Sand-Hill-St-318-Marco-Island-Vacation-Rental-Marco-Island-FL-34145-United-States
[SUCCESS] [05/01/2023, 10:30:07] PID# 274: zillow-672-Western-Lake-Drive-672-Western-Lake-Drive-Santa-Rosa-Beach-FL-32459-Unit

...

[SUCCESS] [05/01/2023, 14:27:48] PID# 2699: zillow-381-Santa-Rosa-Boulevard-W415-Fort-Walton-Beach-FL-32548-United-States
[SUCCESS] [05/01/2023, 14:27:54] PID# 2700: zillow-610-Nuevo-Leon-Lane-The-Villages---Nuevo-Leon-Lane-610-The-Villages-FL-32159-United-States
[SUCCESS] [05/01/2023, 14:28:00] PID# 2701: zillow-10254-East-County-Highway-30A-24w-HP-24W-Condo-High-Pointe-Resort-FL-32461-United-States
[SUCCESS] [05/01/2023, 14:28:07] PID# 2702: zillow-900-Gulf-Shore-Drive-ST1052-Destin-FL-32541-United-States
[SUCCESS] [05/01/2023, 14:28:13] PID# 2703: zillow-1000-Gulf-Blvd-RC308-Indian-Rocks-Beach-FL-33785-United-States
[SUCCESS] [05/01/2023, 14:28:19] PID# 2704: zillow-630-Muirfield-Loop-W301J--1116-Kissimmee-FL-34747-United-States
[SUCCESS] [05/01/2023, 14:28:25] PID# 2705: zillow-909-Santa-Rosa-Blvd-Unit-443-443-El-Matador-Fort-Walton-Beach-FL-32548-United-States
[SUCCESS] [05/01/2023, 14:28:31] PID# 2706: zillow-13720-Gulf-Boulevard-VM305-Madeira-Beach-FL-33708-United-States
[SUCCES

[SUCCESS] [05/01/2023, 14:34:24] PID# 2764: zillow-12-Tarpon-Drive-Surf-Crest-Village-12-St-Augustine-Beach-FL-32080-United-States
[SUCCESS] [05/01/2023, 14:34:30] PID# 2765: zillow-184-E-Water-Street-RB-BLUE3-Rosemary-Beach-FL-32461-United-States
[SUCCESS] [05/01/2023, 14:34:36] PID# 2766: zillow-24-Bourne-Lane-RB-UNCO1-Rosemary-Beach-FL-32461-United-States
[SUCCESS] [05/01/2023, 14:34:42] PID# 2767: zillow-2996-Scenic-Hwy-98-The-Inn-At-Crystal-Beach-#101b-Destin-FL-32541-United-States
[SUCCESS] [05/01/2023, 14:34:48] PID# 2768: zillow-8377-County-Highway-30-A-E-Unit-203-WATER203-Seacrest-FL-32461-United-States
[SUCCESS] [05/01/2023, 14:34:54] PID# 2769: zillow-82-Barrett-Square-S-4A-ROSEM4A-Rosemary-Beach-FL-32461-United-States
[SUCCESS] [05/01/2023, 14:35:00] PID# 2770: zillow-600-Neapolitan-Way-329-Park-Shore-Resort-3rd-Floor-BldgE-End-Unit-Naples-FL-34103-United-States
[SUCCESS] [05/01/2023, 14:35:06] PID# 2771: zillow-505-Old-Beach-Road-BM505OB-Santa-Rosa-Beach-FL-32459-United-St

[SUCCESS] [05/01/2023, 14:41:13] PID# 2830: zillow-3075-NW-79th-Avenue-RL-Grotto-Park-2---3075-NW-79th-Avenue-Ocala-FL-34482-United-States
[SUCCESS] [05/01/2023, 14:41:19] PID# 2831: zillow-1413-Beach-Cottages1413-South-Seas-Beach-Cottage-1413-Captiva-Island-FL-33924-United-States
[SUCCESS] [05/01/2023, 14:41:25] PID# 2832: zillow-16828-Captiva-Drive-Captiva-Paradise-Captiva-Island-FL-33924-United-States
[SUCCESS] [05/01/2023, 14:41:32] PID# 2833: zillow-600-Estero-Blvd-Unit-105-Cane-Palm-Beach-Club-105-Fort-Myers-Beach-FL-33931-United-States
[SUCCESS] [05/01/2023, 14:41:38] PID# 2834: zillow-4450-Estero-Blvd-Unit-403-Villa-Del-Mar-403-Fort-Myers-Beach-FL-33931-United-States
[SUCCESS] [05/01/2023, 14:41:44] PID# 2835: zillow-5115-Gulf-Drive-909-Seychelles-Panama-City-Beach-FL-32408-United-States
[SUCCESS] [05/01/2023, 14:41:50] PID# 2836: zillow-10332-Gulf-Blvd-2-2---Las-Casitas-Treasure-Island-FL-33706-United-States
[SUCCESS] [05/01/2023, 14:41:57] PID# 2837: zillow-381-Santa-Rosa-Bou

[SUCCESS] [05/01/2023, 14:47:57] PID# 2895: zillow-11526-Laika-Lane-RL-Bay-Breeze-Captiva-Island-FL-33924-United-States
[SUCCESS] [05/01/2023, 14:48:04] PID# 2896: zillow-8474-NW-41st-Loop-A-RL-St-Andrews-A---8474-NW-41st-Loop-A-Ocala-FL-34482-United-States
[SUCCESS] [05/01/2023, 14:48:10] PID# 2897: zillow-27-Urchin-Court-Sunset-Captiva-27-Captiva-Island-FL-33924-United-States
[SUCCESS] [05/01/2023, 14:48:16] PID# 2898: zillow-5801-Bahia-Del-Mar-Circle-#314-Unit-314-Bahia-Vista-8-314-St-Petersburg-FL-33715-United-States
[SUCCESS] [05/01/2023, 14:48:23] PID# 2899: zillow-4000-Santa-Maria-Drive-Building-48-#108-Santa-Maria-Holiday-Home-Kissimmee-FL-34741-United-States
[SUCCESS] [05/01/2023, 14:48:29] PID# 2900: zillow-15200-Emerald-Coast-Parkway-Silver-Shells-St-Maarten-304-SSMAARTEN304-Destin-Florida-32541-United-States
[SUCCESS] [05/01/2023, 14:48:35] PID# 2901: zillow-1048-US-98-Silver-Beach-Towers-W1703-SBTW1703-Destin-Florida-32541-United-States
[SUCCESS] [05/01/2023, 14:48:41] PID

[SUCCESS] [05/01/2023, 14:54:25] PID# 2960: zillow-1169-Beach-Walker-Road-1169-Beach-Walker-Fernandina-Beach-FL-32034-United-States
[SUCCESS] [05/01/2023, 14:54:31] PID# 2961: zillow-775-Gulf-Shore-Drive-SPC8147-Destin-FL-32541-United-States
[SUCCESS] [05/01/2023, 14:54:37] PID# 2962: zillow-506-Gulf-Shore-Drive-IR405-Destin-FL-32541-United-States
[SUCCESS] [05/01/2023, 14:54:43] PID# 2963: zillow-4893-Romeo-Circle-4893-Romeo-Circle-Kissimmee-FL-34746-United-States
[SUCCESS] [05/01/2023, 14:54:49] PID# 2964: zillow-154-Gulf-Aire-Drive-Healing-Hideaway-St-Joe-Beach-FL-32456-United-States
[SUCCESS] [05/01/2023, 14:54:55] PID# 2965: zillow-451-Novi-Path-EC025---MND---0615--NLM-Kissimmee-FL-34747-United-States
[SUCCESS] [05/01/2023, 14:55:01] PID# 2966: zillow-230-Burma-St-EC298---STMAR---0219-Kissimmee-FL-34747-United-States
[SUCCESS] [05/01/2023, 14:55:07] PID# 2967: zillow-7466-Bayshore-Drive-LE8-301-Treasure-Island-FL-33706-United-States
[SUCCESS] [05/01/2023, 14:55:13] PID# 2968: zill